Por qué DuckDB
Tu proyecto es local, usando Jupyter Notebook y dos archivos que aunque grandes, caben en un solo equipo.
Simplicidad: DuckDB se instala con pip, no necesita configurar un clúster ni SparkSession.
Parquet friendly: DuckDB lee Parquet directamente, columnas por columnas, y soporta SQL completo para joins, filtros y agregaciones.
Performance excelente en joins y merges de archivos medianos-grandes, sin necesidad de manejar Spark.
Integración con Pandas: puedes traer directamente el resultado a un DataFrame para análisis y visualización, como lo hacemos con fetchdf().

In [1]:
import duckdb
from pyproj import Transformer

In [2]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np

In [3]:
import os
print(os.getcwd())

/Users/eaha/Documents/TFM/mlops-repo/notebooks/02.Transformation


In [4]:
inmuebles_path = "/Users/eaha/Documents/TFM/mlops-repo/data/processed/inmuebles24_departamentos_coordenadas_2025-09-24.parquet"
ageb_path = "/Users/eaha/Documents/TFM/AGEB_fm.csv"
#output_path_aprox = "/Users/eaha/Documents/TFM/mlops-repo/data/processed/df_merged_proximity.parquet"
output_path_nearest = "/Users/eaha/Documents/TFM/mlops-repo/data/processed/df_merged_nearest.parquet"

In [5]:
# ============================================
# CONVERSIÓN DE COORDENADAS INEGI (CCL ITRF2008) → WGS84
# Agrega columnas nuevas a tu CSV original
# ============================================

import pandas as pd
from pyproj import Transformer

# 2️⃣ --- Leer el CSV original ---
df_ageb = pd.read_csv(ageb_path, low_memory=False)
print(f"✅ Archivo cargado: {df_ageb.shape[0]:,} filas, {df_ageb.shape[1]} columnas")

# 3️⃣ --- Crear el transformador ---
# INEGI usa: Cónica Conforme de Lambert (CCL) + Datum ITRF2008 = EPSG:6372
transformer = Transformer.from_crs("EPSG:6372", "EPSG:4326", always_xy=True)

# 4️⃣ --- Validar columnas ---
if not {"lon", "lat"}.issubset(df_ageb.columns):
    raise ValueError("❌ El archivo debe tener columnas llamadas 'lon' y 'lat'.")

# 5️⃣ --- Filtrar filas válidas ---
mask_valid = df_ageb["lon"].notna() & df_ageb["lat"].notna()

# 6️⃣ --- Transformar coordenadas ---
lon_wgs84, lat_wgs84 = transformer.transform(
    df_ageb.loc[mask_valid, "lon"].astype(float).values,
    df_ageb.loc[mask_valid, "lat"].astype(float).values
)

# 7️⃣ --- Crear nuevas columnas (rellenar solo las válidas) ---
df_ageb["lon_wgs84"] = pd.NA
df_ageb["lat_wgs84"] = pd.NA
df_ageb.loc[mask_valid, "lon_wgs84"] = lon_wgs84
df_ageb.loc[mask_valid, "lat_wgs84"] = lat_wgs84

# 8️⃣ --- Vista previa de resultados ---
print("\n🔍 Ejemplo de coordenadas transformadas:")
print(df_ageb[["lon", "lat", "lon_wgs84", "lat_wgs84"]].head(10))

# 9️⃣ --- Guardar el CSV actualizado ---
# 🔁 Puedes sobrescribir el original o crear una copia nueva (recomendado)
output_path = "/Users/eaha/Documents/TFM/AGEB_fm_wgs84.csv"
df_ageb.to_csv(output_path, index=False, encoding="utf-8")

print(f"\n✅ Archivo guardado con nuevas columnas en: {output_path}")
print("   Columnas agregadas: 'lon_wgs84', 'lat_wgs84'")


✅ Archivo cargado: 375,642 filas, 128 columnas

🔍 Ejemplo de coordenadas transformadas:
            lon            lat  lon_wgs84  lat_wgs84
0           NaN            NaN       <NA>       <NA>
1           NaN            NaN       <NA>       <NA>
2           NaN            NaN       <NA>       <NA>
3           NaN            NaN       <NA>       <NA>
4  2.792381e+06  837760.851800 -99.205885  19.512833
5  2.792330e+06  837886.666029 -99.206353  19.513982
6  2.792384e+06  837831.967902  -99.20585  19.513477
7  2.792340e+06  837819.149608 -99.206271  19.513368
8  2.792369e+06  837907.005180 -99.205974  19.514159
9  2.792420e+06  837841.365701 -99.205502  19.513555

✅ Archivo guardado con nuevas columnas en: /Users/eaha/Documents/TFM/AGEB_fm_wgs84.csv
   Columnas agregadas: 'lon_wgs84', 'lat_wgs84'


In [6]:
output_path = "/Users/eaha/Documents/TFM/AGEB_fm_wgs84.csv"

In [7]:
#Crear la conexión a DuckDB (en memoria)
# ---------------------------------------------------
con = duckdb.connect(database=':memory:')  # base temporal en memoria

# Instalar la extensión spatial (solo la primera vez)
#con.execute("INSTALL spatial;")

# Cargar la extensión en esta sesión
con.execute("LOAD spatial;")

In [8]:
# all_varchar=True evita errores por columnas mixtas
con.execute(f"""
CREATE TABLE ageb AS
SELECT *
FROM read_csv_auto('{output_path}', all_varchar=True, ignore_errors=True)
""")

# Convertir columnas lat y lon a DOUBLE y renombrar
con.execute("""
ALTER TABLE ageb
ALTER COLUMN lat_wgs84 TYPE DOUBLE
""")
con.execute("""
ALTER TABLE ageb
ALTER COLUMN lon_wgs84 TYPE DOUBLE
""")
con.execute("""
ALTER TABLE ageb
RENAME COLUMN lat_wgs84 TO latitud
""")
con.execute("""
ALTER TABLE ageb
RENAME COLUMN lon_wgs84 TO longitud
""")

In [9]:
con.execute(f"""
CREATE TABLE inmuebles AS
SELECT *
FROM read_parquet('{inmuebles_path}')
WHERE latitud IS NOT NULL
  AND longitud IS NOT NULL
""")

# Asegurar que latitud y longitud sean DOUBLE
con.execute("""
ALTER TABLE inmuebles
ALTER COLUMN latitud TYPE DOUBLE
""")
con.execute("""
ALTER TABLE inmuebles
ALTER COLUMN longitud TYPE DOUBLE
""")

Al eliminar las filas cuya latitud o longitud contenian NaN del dataset de inmuebles, pasé de 10233 filas a 7206

In [10]:
# ---------------------------------------------------
# Merge aproximado por cercanía geográfica
# ---------------------------------------------------
# Aproximación: 1 grado ~ 111 km, 50 m ≈ 0.00045 grados
tolerance_degrees = 0.00045

In [11]:
query_nearest_ageb = f"""
WITH inmuebles_id AS (
    SELECT
        *,
        ROW_NUMBER() OVER () AS id_inmueble
    FROM inmuebles
),
joined AS (
    SELECT 
        i.*, 
        a.*, 
        ST_Distance(
            ST_Point(i.longitud, i.latitud),
            ST_Point(a.longitud, a.latitud)
        ) AS distancia,
        ROW_NUMBER() OVER (
            PARTITION BY i.id_inmueble 
            ORDER BY ST_Distance(
                ST_Point(i.longitud, i.latitud),
                ST_Point(a.longitud, a.latitud)
            )
        ) AS rn
    FROM inmuebles_id i
    LEFT JOIN ageb a
        ON ST_Distance(
            ST_Point(i.longitud, i.latitud),
            ST_Point(a.longitud, a.latitud)
        ) <= {tolerance_degrees}
)
SELECT *
FROM joined
WHERE rn = 1
"""

In [12]:
df_merged_nearest = con.execute(query_nearest_ageb).fetchdf()

In [13]:
df_merged_nearest.to_parquet(output_path_nearest, index=False)

In [14]:
print("Merged DataFrame (nearest) shape:", df_merged_nearest.shape)
df_merged_nearest.head()

Merged DataFrame (nearest) shape: (7206, 160)


,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,cuenta_con_cocina_integral,cuenta_con_sala,cuenta_con_closet,...,CVESEG3,CVEREF3,TIPOVR3,NOMREF3,lon,lat,longitud_1,latitud_2,distancia,rn
0,24000.0,100,2.0,2.0,1.0,0,0,1,1,0,...,41.0,3.0,Avenida,Luis Cabrera,2791194.699523996,816629.8927558392,-99.221135,19.321632,0.000224,1
1,26000.0,89,2.0,2.0,2.0,0,0,1,1,0,...,6.0,2.0,Calle,Luis Moya,2798797.8143767924,828996.7264598252,-99.146250,19.432296,0.000395,1
2,53000.0,125,2.0,2.0,1.0,1,0,1,1,0,...,5.0,4.0,Calle,Hamburgo,2797164.8937426186,828620.444331886,-99.161913,19.429181,0.000406,1
3,42000.0,80,2.0,2.0,1.0,1,0,0,1,1,...,5.0,1.0,Calle,Zacatecas,2797566.5262409304,827090.9618393083,-99.158367,19.415255,0.000351,1
4,18000.0,60,1.0,1.0,1.0,0,0,1,0,0,...,2.0,1.0,Calle,Francisco Pimentel,2796997.1725512976,829553.368652961,-99.163338,19.437662,0.000399,1


In [15]:
df_merged_nearest = pd.read_parquet("/Users/eaha/Documents/TFM/mlops-repo/data/processed/df_merged_nearest.parquet")
df_inmuebles = pd.read_parquet(inmuebles_path)

In [16]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

subset = df_merged_nearest[['latitud', 'longitud', 'latitud_2', 'longitud_1']].head(1000)
print(subset)

pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

       latitud    longitud  latitud_2  longitud_1
0    19.321482  -99.220969  19.321632  -99.221135
1    19.432691  -99.146258  19.432296  -99.146250
2    19.428833  -99.162121  19.429181  -99.161913
3    19.415205  -99.158714  19.415255  -99.158367
4    19.437731  -99.163730  19.437662  -99.163338
5    19.417732  -99.174422  19.417640  -99.174220
6    19.431345  -99.169125  19.431223  -99.169430
7    19.434764  -99.144095  19.434549  -99.143717
8    19.434858  -99.152343        NaN         NaN
9    19.435859  -99.157079        NaN         NaN
10   19.409043  -99.148487  19.409040  -99.148479
11   19.409480  -99.175115  19.409330  -99.175019
12   19.412170  -99.163667  19.412162  -99.163746
13   19.430172  -99.168479  19.430259  -99.168184
14   19.406270  -99.171275  19.406265  -99.171343
15   19.410579  -99.170625  19.410420  -99.170623
16   19.434251  -99.154816  19.434273  -99.154654
17   19.411636  -99.154732        NaN         NaN
18   19.422151  -99.169483  19.421898  -99.169492


In [17]:
# Contar NaN en columnas específicas
na_counts = df_merged_nearest[['latitud_2', 'longitud_1']].isna().sum()

print("Conteo de NaN:")
print(na_counts)

Conteo de NaN:
latitud_2     1279
longitud_1    1279
dtype: int64


In [18]:
na_percent = (na_counts / len(df_merged_nearest)) * 100
print("Porcentaje de NaN:")
print(na_percent)

Porcentaje de NaN:
latitud_2     17.749098
longitud_1    17.749098
dtype: float64


In [10]:
# ---------------------------------------------------
# Merge aproximado por cercanía geográfica
# ---------------------------------------------------
# Aproximación: 1 grado ~ 111 km, 50 m ≈ 0.00045 grados
tolerance_degrees = 0.00045

query_proximity = f"""
SELECT i.*, a.*
FROM inmuebles i
LEFT JOIN ageb a
ON ST_Distance(
       ST_Point(i.longitud, i.latitud),
       ST_Point(a.longitud, a.latitud)
   ) <= {tolerance_degrees}
"""

df_merged_proximity = con.execute(query_proximity).fetchdf()

In [12]:
df_merged_proximity.to_parquet(output_path_aprox, index=False)

In [13]:
print("Merged DataFrame (proximity) shape:", df_merged_proximity.shape)
df_merged_proximity.head()

Merged DataFrame (proximity) shape: (20110, 157)


,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,cuenta_con_cocina_integral,cuenta_con_sala,cuenta_con_closet,...,NOMREF2,CVEVIAL3,CVESEG3,CVEREF3,TIPOVR3,NOMREF3,lon,lat,longitud_1,latitud_1
0,6500.0,70,3.0,1.0,1.0,1,0,0,0,1,...,Ingenieros,None,None,0.0,None,None,2796303.591669979,825486.1295704067,-99.170727,19.400944
1,24000.0,110,1.0,1.0,1.0,1,0,0,0,0,...,Gaviota,652.0,5.0,1.0,Calle,Arquitecto Carlos Lazo,2794257.5451484043,825511.1022766106,-99.190254,19.401535
2,24000.0,110,1.0,1.0,1.0,1,0,0,0,0,...,Cerrada Manuel Dublan,1238.0,1.0,5.0,Calle,Cerrada Rufina,2794271.7812266974,825507.9726308832,-99.190119,19.401504
3,21500.0,75,2.0,2.0,1.0,0,0,0,1,1,...,José Martí,493.0,3.0,4.0,Calle,Comercio,2795638.0385,825689.4462000001,-99.177042,19.402905
4,21500.0,75,2.0,2.0,1.0,0,0,0,1,1,...,Avenida Progreso,490.0,3.0,2.0,Calle,Minería,2795646.5672624097,825686.6095061364,-99.176961,19.402877


In [14]:
df_merged_proximity = pd.read_parquet("/Users/eaha/Documents/TFM/mlops-repo/data/processed/df_merged_proximity.parquet")
df_inmuebles = pd.read_parquet(inmuebles_path)

In [15]:
df_merged_proximity.shape

(20110, 157)

In [16]:
df_inmuebles.shape

(10233, 27)

Al eliminar las filas cuya latitud o longitud contenian NaN, pasé de 10233 filas a 7206

In [17]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

subset = df_merged_proximity[['latitud', 'longitud', 'latitud_1', 'longitud_1']].head(1000)
print(subset)

pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

       latitud   longitud  latitud_1  longitud_1
0    19.401118 -99.170508  19.400944  -99.170727
1    19.401579 -99.190055  19.401535  -99.190254
2    19.401579 -99.190055  19.401504  -99.190119
3    19.402525 -99.177142  19.402905  -99.177042
4    19.402525 -99.177142  19.402877  -99.176961
5    19.397068 -99.231656  19.397094  -99.231619
6    19.398535 -99.177557  19.398519  -99.177676
7    19.398535 -99.177557  19.398535  -99.177547
8    19.401579 -99.190055  19.401535  -99.190254
9    19.401579 -99.190055  19.401504  -99.190119
10   19.406497 -99.186058  19.406755  -99.186088
11   19.406497 -99.186058  19.406751  -99.186047
12   19.393108 -99.239439  19.393522  -99.239421
13   19.393108 -99.239439  19.392874  -99.239535
14   19.404771 -99.179552  19.404745  -99.179890
15   19.404771 -99.179552  19.404812  -99.179869
16   19.387011 -99.243636  19.387128  -99.243354
17   19.405852 -99.184884  19.406261  -99.184906
18   19.405852 -99.184884  19.405773  -99.185272
19   19.405852 -99.1

In [18]:
# Contar NaN en columnas específicas
na_counts = df_merged_proximity[['latitud_1', 'longitud_1']].isna().sum()

print("Conteo de NaN:")
print(na_counts)

Conteo de NaN:
latitud_1     1279
longitud_1    1279
dtype: int64


In [19]:
na_percent = (na_counts / len(df_merged_proximity)) * 100
print("Porcentaje de NaN:")
print(na_percent)


Porcentaje de NaN:
latitud_1     6.36002
longitud_1    6.36002
dtype: float64


Al parecer ya lo logré y tengo solo una perdida del 6.37% de mis datos

Al parecer estoy usando un umbral muy pequeño, entonces voy a encontrar un umbral razonable para hacer mis joins

In [41]:
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")


In [42]:
# 2 Crear tabla temporal con id único para cada inmueble
con.execute("""
CREATE OR REPLACE TABLE inmuebles_id AS
SELECT
  ROW_NUMBER() OVER () AS id_inmueble,
  latitud,
  longitud
FROM inmuebles
""")

In [44]:
# 3 Calcular la distancia mínima (en metros) al AGEB más cercano
con.execute("""
CREATE OR REPLACE TABLE distancias_minimas AS
SELECT 
  i.id_inmueble,
  MIN(ST_Distance_Sphere(ST_Point(i.longitud, i.latitud),
                        ST_Point(a.longitud, a.latitud))) AS distancia_m
FROM inmuebles_id i
CROSS JOIN ageb a
GROUP BY i.id_inmueble
""")

In [45]:
distancias = con.execute("SELECT distancia_m FROM distancias_minimas").fetchnumpy()["distancia_m"]

In [46]:
percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
valores = np.nanpercentile(distancias, percentiles)

In [47]:
print("📏 Percentiles de la distancia mínima a un AGEB (en metros):\n")
for p, v in zip(percentiles, valores):
    print(f"  {p:>3}%  →  {v:8.2f} m")

📏 Percentiles de la distancia mínima a un AGEB (en metros):

    1%  →    658.74 m
    5%  →    958.76 m
   10%  →   1147.59 m
   25%  →   1850.20 m
   50%  →   2630.15 m
   75%  →   4407.59 m
   90%  →   7209.30 m
   95%  →   7819.18 m
   99%  →  11765.65 m


In [50]:
# Elegir el umbral (metros)
tolerance_m = 2000

# Ejecutar el merge espacial
df_merged = con.execute(f"""
SELECT 
    i.*, 
    a.*
FROM inmuebles i
LEFT JOIN ageb a
ON ST_Distance_Sphere(ST_Point(i.longitud, i.latitud),
                     ST_Point(a.longitud, a.latitud)) <= {tolerance_m}
""").fetchdf()

# Guardar resultado
df_merged.to_parquet("/Users/eaha/Documents/TFM/mlops-repo/data/processed/merged_inmuebles_ageb_2000.parquet", index=False)

print(f"✅ Merge completado con tolerancia de {tolerance_m} m")
print(f"📦 Filas resultantes: {len(df_merged)}")


✅ Merge completado con tolerancia de 2000 m
📦 Filas resultantes: 7206


In [55]:
df_merged_inm_2000 = pd.read_parquet("/Users/eaha/Documents/TFM/mlops-repo/data/processed/merged_inmuebles_ageb_2000.parquet")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

subset = df_merged_inm_2000[['latitud', 'longitud', 'latitud_1', 'longitud_1']].head(100)
print(subset)

pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

      latitud   longitud      latitud_1    longitud_1
0   19.418946 -99.175008  812359.472562  2.808999e+06
1   19.419490 -99.163997  812359.472562  2.808999e+06
2   19.434492 -99.169216  812359.472562  2.808999e+06
3   19.430966 -99.158874  812359.472562  2.808999e+06
4   19.405035 -99.174838  812359.472562  2.808999e+06
5   19.417428 -99.177700  812359.472562  2.808999e+06
6   19.420006 -99.164805  812359.472562  2.808999e+06
7   19.430383 -99.168114  812359.472562  2.808999e+06
8   19.415786 -99.163703  812359.472562  2.808999e+06
9   19.430966 -99.158874  812359.472562  2.808999e+06
10  19.423379 -99.160647  812359.472562  2.808999e+06
11  19.427519 -99.167912  812359.472562  2.808999e+06
12  19.412464 -99.171079  812359.472562  2.808999e+06
13  19.400854 -99.169164  812359.472562  2.808999e+06
14  19.415102 -99.169527  812359.472562  2.808999e+06
15  19.415102 -99.169527  812359.472562  2.808999e+06
16  19.412120 -99.172986  812359.472562  2.808999e+06
17  19.412120 -99.172986  81

In [58]:
df_merged_inm24 = pd.read_parquet("/Users/eaha/Documents/TFM/mlops-repo/data/processed/inmuebles24_departamentos_coordenadas_2025-09-24.parquet")
df_merged_inm24.head()

,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,cuenta_con_cocina_integral,cuenta_con_sala,cuenta_con_closet,...,cuenta_con_area_de_lavado,cuenta_con_salon_usos_multiples,cuenta_con_mantenimiento_incluido,cuenta_con_vigilancia_24_horas,direccion,colonia,cp,municipio,latitud,longitud
0,24000.0,100,2.0,2.0,1.0,0,0,1,1,0,...,0,0,0,0,san jeronimo lidice san jeronimo lidice la mag...,san jeronimo lidice,10200,la magdalena contreras,19.321482,-99.220969
1,12000.0,50,1.0,1.0,1.0,1,0,0,1,1,...,1,0,1,0,callejon del prado barrio san francisco la mag...,el prado,9480,la magdalena contreras,NaN,NaN
2,34100.0,232,3.0,4.0,2.0,0,0,1,1,1,...,0,0,0,1,blvd. adolfo ruiz cortines 2775 san jeronimo l...,adolfo ruiz cortines,4630,la magdalena contreras,19.319711,-99.221621
3,16000.0,165,2.0,1.0,1.0,0,0,0,1,0,...,0,0,0,0,magnolia 26 san jeronimo lidice la magdalena c...,san jeronimo lidice,10200,la magdalena contreras,19.318938,-99.228502
4,26000.0,180,3.0,2.0,2.0,0,0,1,0,1,...,0,0,0,1,san marcos 11 pedregal 2 la magdalena contreras,san marcos,2020,la magdalena contreras,19.307883,-99.228310


In [62]:
ageb_fm = pd.read_csv(ageb_path)
ageb_fm.head(100)

/var/folders/5x/v8hk2rj9117bkb38sq4yjyfw0000gn/T/ipykernel_53946/3956344545.py:1: DtypeWarning: Columns (6,50,104,111,112,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  ageb_fm = pd.read_csv(ageb_path)


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,CVEREF2,TIPOVR2,NOMREF2,CVEVIAL3,CVESEG3,CVEREF3,TIPOVR3,NOMREF3,lon,lat
0,9,Ciudad de México,0,Total de la entidad Ciudad de México,0,Total de la entidad,0000,0,9209944,4805017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,Ciudad de México,2,Azcapotzalco,0,Total del municipio,0000,0,432205,227255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,Ciudad de México,2,Azcapotzalco,1,Total de la localidad urbana,0000,0,432205,227255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,Ciudad de México,2,Azcapotzalco,1,Total AGEB urbana,0010,0,3183,1695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,Ciudad de México,2,Azcapotzalco,1,Azcapotzalco,0010,1,159,86,...,2.0,Calle,Física,1618.0,3.0,1.0,Avenida,Civilizaciones,2.792381e+06,837760.851800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9,Ciudad de México,2,Azcapotzalco,1,Azcapotzalco,0010,28,94,52,...,4.0,Avenida,Calzada de las Armas Norte,1611.0,1.0,3.0,Calle,Geografía,2.792055e+06,837628.188815
96,9,Ciudad de México,2,Azcapotzalco,1,Azcapotzalco,0010,28,94,52,...,3.0,Calle,Geografía,1610.0,2.0,4.0,Avenida,Calzada de las Armas Norte,2.792066e+06,837583.959500
97,9,Ciudad de México,2,Azcapotzalco,1,Azcapotzalco,0010,28,94,52,...,4.0,Avenida,Calzada de las Armas Norte,1617.0,1.0,1.0,Calle,Cibernética,2.792025e+06,837597.747430
98,9,Ciudad de México,2,Azcapotzalco,1,Azcapotzalco,0010,28,94,52,...,3.0,Calle,Geografía,18.0,16.0,2.0,Calle,Herreros,2.792014e+06,837641.976939
